<a href="https://colab.research.google.com/github/SushrutReddy/DL-ASSiGNMENT-2/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense


In [3]:
# Function to read and process transliteration data
def load_data(file_path):
    df = pd.read_csv(file_path, sep='\t', header=None).dropna()
    return [(row[1], row[0]) for _, row in df.iterrows()]  # Latin, Devanagari pairs

# Load the train and validation data
train_pairs = load_data('hi.translit.sampled.train.tsv')
val_pairs = load_data('hi.translit.sampled.dev.tsv')

# Prepare sequences with start/end tokens
input_texts = [src for src, _ in train_pairs]
target_texts = ['\t' + tgt + '\n' for _, tgt in train_pairs]

# Create character sets for input and output
input_chars = sorted(set(''.join(input_texts)))
target_chars = sorted(set(''.join(target_texts)))

# Create token-to-index mappings
input_token_index = {ch: i for i, ch in enumerate(input_chars)}
target_token_index = {ch: i for i, ch in enumerate(target_chars)}

# Determine the max sequence lengths
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

# Initialize data arrays for one-hot encoding
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, len(input_chars)), dtype='float32')
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, len(target_chars)), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, len(target_chars)), dtype='float32')

# Fill one-hot encoded arrays for input and target sequences
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0


In [4]:
# Encoder model setup
encoder_inputs = Input(shape=(None, len(input_chars)))
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder model setup
decoder_inputs = Input(shape=(None, len(target_chars)))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(target_chars), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define and compile the full model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 26)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 65)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    289,792 │ input_layer[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    329,728 │ input_layer_1[0]… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 65)  │     16,705 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 636,225 (2.43 MB)

 Trainable params: 636,225 (2.43 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Train the model
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=90,
    validation_split=0.22
)


Epoch 1/90
539/539 ━━━━━━━━━━━━━━━━━━━━ 114s 211ms/step - accuracy: 0.0668 - loss: 1.1488 - val_accuracy: 0.0625 - val_loss: 1.2700
Epoch 2/90
539/539 ━━━━━━━━━━━━━━━━━━━━ 144s 215ms/step - accuracy: 0.0806 - loss: 1.1031 - val_accuracy: 0.0598 - val_loss: 1.2844
Epoch 3/90
539/539 ━━━━━━━━━━━━━━━━━━━━ 115s 214ms/step - accuracy: 0.0960 - loss: 1.0503 - val_accuracy: 0.0595 - val_loss: 1.2894
Epoch 4/90
539/539 ━━━━━━━━━━━━━━━━━━━━ 152s 233ms/step - accuracy: 0.1089 - loss: 1.0147 - val_accuracy: 0.0576 - val_loss: 1.2804
Epoch 5/90
539/539 ━━━━━━━━━━━━━━━━━━━━ 142s 234ms/step - accuracy: 0.1146 - loss: 0.9890 - val_accuracy: 0.0597 - val_loss: 1.2990
Epoch 6/90
539/539 ━━━━━━━━━━━━━━━━━━━━ 115s 214ms/step - accuracy: 0.1192 - loss: 0.9695 - val_accuracy: 0.0627 - val_loss: 1.2793
Epoch 7/90
539/539 ━━━━━━━━━━━━━━━━━━━━ 142s 215ms/step - accuracy: 0.1236 - loss: 0.9569 - val_accuracy: 0.0600 - val_loss: 1.2735
Epoch 8/90
539/539 ━━━━━━━━━━━━━━━━━━━━ 141s 214ms/step - accuracy: 0.1268 -

In [9]:
# Save the encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Define decoder model for inference
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Reverse lookup for character indices
reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}


In [10]:
# Function to decode sequence (input to target)
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, len(target_chars)))
    target_seq[0, 0, target_token_index['\t']] = 1.0

    decoded_sentence = ''
    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n':
            break

        target_seq = np.zeros((1, 1, len(target_chars)))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_sentence.strip()


In [11]:
# Test the model on the first 10 sequences
for seq_index in range(10):
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded = decode_sequence(input_seq)
    print(f"Input: {input_texts[seq_index]} → Prediction: {decoded} | Target: {target_texts[seq_index]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 591ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Input: an → Prediction: अ | Target: 	अं

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Input: ankganit → Prediction: अंंकाा | Target: 	अंकगणित

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Input: uncle → Prediction: अंं | Target: 	अंकल

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Input: ankur → Pr

In [12]:
# Function to prepare input text for prediction
def encode_input_text(input_text):
    encoder_input = np.zeros((1, max_encoder_seq_length, len(input_chars)))
    for t, char in enumerate(input_text):
        if char in input_token_index:
            encoder_input[0, t, input_token_index[char]] = 1.0
    return encoder_input

# Interactive loop for user input
while True:
    user_input = input("Enter a Latin word (or type 'exit' to quit): ").strip().lower()
    if user_input == 'exit':
        break

    encoded_input = encode_input_text(user_input)
    prediction = decode_sequence(encoded_input)
    print(f"Predicted Devanagari: {prediction}")


Enter a Latin word (or type 'exit' to quit): ankush
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted Devanagari: अंका
Enter a Latin word (or type 'exit' to quit): anga
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Predicted Devanagari: अं
Enter a Latin word (or type 'exit' to quit): ankurit
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicted Devanagari: अंकुर
Enter a Latin word (or type 'exit' to quit): exit


Question 2

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json when prompted

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d paultimothymooney/poetry
!unzip poetry.zip

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/poetry
License(s): CC0-1.0
Archive:  poetry.zip
  inflating: Kanye_West.txt          
  inflating: Lil_Wayne.txt           
  inflating: adele.txt               
  inflating: al-green.txt            
  inflating: alicia-keys.txt         
  inflating: amy-winehouse.txt       
  inflating: beatles.txt             
  inflating: bieber.txt              
  inflating: bjork.txt               
  inflating: blink-182.txt           
  inflating: bob-dylan.txt           
  inflating: bob-marley.txt          
  inflating: britney-spears.txt      
  inflating: bruce-springsteen.txt   
  inflating: bruno-mars.txt          
  inflating: cake.txt                
  inflating: dickinson.txt           
  inflating: disney.txt              
  inflating: dj-khaled.txt           
  inflating: dolly-parton.txt        
  inflating: dr-seuss.txt            
  inflating: drake.txt               
  inflating: eminem.txt              
  inflating: ja

In [ ]:
!pip install transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Path to the drake.txt file
drake_file_path = './drake.txt'

# Read the file to confirm it's loaded
with open(drake_file_path, 'r', encoding='utf-8') as file:
    drake_lyrics = file.read()

print(drake_lyrics[:500])  # Display the first 500 characters of the lyrics to verify


[Hook]
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face [Verse 1]
Somethin' ain't right when we talkin'
Somethin' ain't right when we talkin'
Look like you hidin' your problems
Really you never was solid
No, you can't "son" me
You won't neve


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Function to load the dataset (drake.txt) and create a text dataset
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

# Function to get the data collator for language modeling
def get_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

# Load the Drake dataset
dataset = load_dataset(drake_file_path, tokenizer)
data_collator = get_data_collator(tokenizer)


/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./gpt2-drake-lyrics",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
    fp16=torch.cuda.is_available(),  # Enable mixed precision if GPU is available
    report_to='none',  # Disable W&B logging
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)


In [ ]:
trainer.train()
trainer.save_model("./gpt2-drake-lyrics")
tokenizer.save_pretrained("./gpt2-drake-lyrics")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.587300
200,3.344600
300,3.140200
400,3.048700
500,2.854600
600,2.805600


('./gpt2-drake-lyrics/tokenizer_config.json',
 './gpt2-drake-lyrics/special_tokens_map.json',
 './gpt2-drake-lyrics/vocab.json',
 './gpt2-drake-lyrics/merges.txt',
 './gpt2-drake-lyrics/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load the trained model and tokenizer for generation
generator = pipeline("text-generation", model="./gpt2-drake-lyrics", tokenizer="./gpt2-drake-lyrics")

# Set your prompt
prompt = "Started from the bottom, now we're here"
generated_lyrics = generator(prompt, max_length=100, num_return_sequences=1)

# Display the generated lyrics
print(generated_lyrics[0]['generated_text'])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Started from the bottom, now we're here
And we've been here before
And he never should've left
It's just not your time
You're still here, still in control of it, still with it, now I'm here And I know you could go to bed this morning But you could sleep better that you're gone
So if you didn't wake up this morning
I don't know what I'm going through
I just don't know what to say



In [ ]:
trainer.train()
# Check loss every 100 steps


Step,Training Loss
100,2.774100
200,2.735800
300,2.559700
400,2.537200
500,2.326900
600,2.300900


TrainOutput(global_step=651, training_loss=2.52590090375159, metrics={'train_runtime': 3802.4943, 'train_samples_per_second': 0.342, 'train_steps_per_second': 0.171, 'total_flos': 84854587392000.0, 'train_loss': 2.52590090375159, 'epoch': 3.0})